In [1]:
!python --version

Python 3.7.13


In [3]:
!pwd
!pip install pyspark


/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 60.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=4e174c457c12902ac0ed098ac8e886a284f1ef61823a5ca1ebaf3099a1942105
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [7]:
sc=SparkSession.builder.appName("PySpark in Google colab Example1").getOrCreate

In [52]:
import numpy as np
from pprint import pprint
from collections import defaultdict

m = np.matrix(
    [ 
        [1, 2, 0],
        [3, 0, 5],
        [0, 7, 0],
        [4, 0, 0],
        [1, 8, 2]
    ]
)

n = np.matrix(
    [
        [1, 4, 5, 0],
        [0, 4, 0, 1],
        [6, 3, 9, 1],
    ]
)

In [53]:
shape_m = m.shape
shape_n = n.shape
pprint(shape_m)
pprint(shape_n)
m_n = m * n
pprint(m_n)

(5, 3)
(3, 4)
matrix([[ 1, 12,  5,  2],
        [33, 27, 60,  5],
        [ 0, 28,  0,  7],
        [ 4, 16, 20,  0],
        [13, 42, 23, 10]])


In [54]:
# Representing matrices in the form of i, j , m[i][j] and i, j, n[i][j]
print(m.shape[0])
print(m.shape[1])
m = [[i, j, m[i,j]] for i in range(m.shape[0]) for j in range(m.shape[1]) if m[i,j] != 0]
pprint(m)

5
3
[[0, 0, 1],
 [0, 1, 2],
 [1, 0, 3],
 [1, 2, 5],
 [2, 1, 7],
 [3, 0, 4],
 [4, 0, 1],
 [4, 1, 8],
 [4, 2, 2]]


In [55]:
n = [[i, j, n[i,j]] for i in range(n.shape[0]) for j in range(n.shape[1]) if n[i, j] != 0]
print(len(n))
pprint(n)

9
[[0, 0, 1],
 [0, 1, 4],
 [0, 2, 5],
 [1, 1, 4],
 [1, 3, 1],
 [2, 0, 6],
 [2, 1, 3],
 [2, 2, 9],
 [2, 3, 1]]


In [56]:
# create key value pairs fot the 1st matrix
# key = j, values = [mij, ...]
ma = defaultdict(list)
pprint(ma)
for j in range(len(m)):
    ma[m[j][1]].append((m[j][0], m[j][2]))

pprint(ma)

defaultdict(<class 'list'>, {})
defaultdict(<class 'list'>,
            {0: [(0, 1), (1, 3), (3, 4), (4, 1)],
             1: [(0, 2), (2, 7), (4, 8)],
             2: [(1, 5), (4, 2)]})


In [57]:
# create key value pairs

# key = j, value = njk
na = defaultdict(list)
for j in range(len(n)):
    na[n[j][0]].append((n[j][1], n[j][2]))

pprint(na)

defaultdict(<class 'list'>,
            {0: [(0, 1), (1, 4), (2, 5)],
             1: [(1, 4), (3, 1)],
             2: [(0, 6), (1, 3), (2, 9), (3, 1)]})


In [58]:
# Reduce keys for each possible j value
# Group by keys from ma, na
# For each j value we will take each i, mij value from ma and multiply it by k, mjk
# The key will now be this (i, k) value
op = defaultdict(list)
for j in range(shape_m[1]):
    if j in ma and j in na:
        for mi in ma[j]:
            for ni in na[j]: 
                i = mi[0]
                k = ni[0]
                op[(i,k)].append(mi[1] * ni[1])
pprint(op)

defaultdict(<class 'list'>,
            {(0, 0): [1],
             (0, 1): [4, 8],
             (0, 2): [5],
             (0, 3): [2],
             (1, 0): [3, 30],
             (1, 1): [12, 15],
             (1, 2): [15, 45],
             (1, 3): [5],
             (2, 1): [28],
             (2, 3): [7],
             (3, 0): [4],
             (3, 1): [16],
             (3, 2): [20],
             (4, 0): [1, 12],
             (4, 1): [4, 32, 6],
             (4, 2): [5, 18],
             (4, 3): [8, 2]})


In [76]:
# Group by the keys again and sum the values to get the final result which is the multiplication of m and n
ans = list()
for k, v in op.items():
    ans.append([k[0], k[1], sum(v)])    
pprint(sorted(ans))


[[0, 0, 1],
 [0, 1, 12],
 [0, 2, 5],
 [0, 3, 2],
 [1, 0, 33],
 [1, 1, 27],
 [1, 2, 60],
 [1, 3, 5],
 [2, 1, 28],
 [2, 3, 7],
 [3, 0, 4],
 [3, 1, 16],
 [3, 2, 20],
 [4, 0, 13],
 [4, 1, 42],
 [4, 2, 23],
 [4, 3, 10]]


In [60]:
# Compare with results obtained using numpy matrix multiplication
m_n = [[i, j, m_n[i,j]] for i in range(m_n.shape[0]) for j in range(m_n.shape[1]) if m_n[i,j] != 0]
pprint(m_n)

[[0, 0, 1],
 [0, 1, 12],
 [0, 2, 5],
 [0, 3, 2],
 [1, 0, 33],
 [1, 1, 27],
 [1, 2, 60],
 [1, 3, 5],
 [2, 1, 28],
 [2, 3, 7],
 [3, 0, 4],
 [3, 1, 16],
 [3, 2, 20],
 [4, 0, 13],
 [4, 1, 42],
 [4, 2, 23],
 [4, 3, 10]]


TypeError: ignored